# Notebook: Explore Addressed Topics

This notebook is used to explore the topics discussed in reviews previously discussed. 

## Packages

In [1]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP
import pandas as pd
import spacy
import nltk
import re

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/Caskroom/min

## Constants

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
DATASET_PATH = "reviews_dataset/balanced_reviews_sentences.csv"
STOPWORDS = set(stopwords.words('german'))
N_TOPICS = 6

## Code

In [4]:
#spacy.cli.download("de_core_news_sm")

In [5]:
nlp = spacy.load('de_core_news_sm')

In [6]:
dataset = pd.read_csv(DATASET_PATH)

In [7]:
def clean_text(text):
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)

    text = re.sub(r'@\w+', '', text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
    return text.strip()

def lemmatize_remove_stopwords_text(text):
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_.lower() for token in doc if token.text.lower() not in STOPWORDS])
    return lemmatized_text

dataset["text"] = dataset["text"].apply(clean_text)
dataset["text"] = dataset["text"].apply(lemmatize_remove_stopwords_text)

In [8]:
dataset = dataset.text.to_list()

In [9]:
# Use default UMAP but add random state
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=43)
model = BERTopic(language="german", nr_topics=N_TOPICS, umap_model=umap_model, top_n_words=20, embedding_model="paraphrase-multilingual-MiniLM-L12-v2")

In [10]:
topics, probabilities = model.fit_transform(dataset)
topics = model.reduce_outliers(dataset, topics, strategy="distributions")
model.update_topics(dataset, topics, top_n_words=20)

In [11]:
model.get_document_info(dataset)["Topic"].value_counts().reset_index()

,Topic,count
0,0,3710
1,1,103
2,2,69
3,3,56
4,4,50
5,-1,12


In [12]:
topic_statistics = model.get_document_info(dataset)["Topic"].value_counts().reset_index()#.rename(columns={"index": "Topic", "Topic": "Frequency"}).sort_values("Topic").reset_index(drop=True)
topic_statistics = topic_statistics[topic_statistics["Topic"] != -1]
topic_statistics["Topic"] = topic_statistics["Topic"] + 1
topic_statistics

,Topic,count
0,1,3710
1,2,103
2,3,69
3,4,56
4,5,50


In [13]:
top_words = []
for topic in topic_statistics["Topic"]:
    words = model.get_topic(topic - 1)[:5]
    top_words.append(', '.join([word[0] for word in words]))
    
topic_statistics["Top Words"] = top_words

def format_frequency(frequency):
    return "{:,}".format(frequency)

# Anwendung der Funktion auf die Spalte "Frequency"
topic_statistics["count"] = topic_statistics["count"].apply(format_frequency)

topic_statistics

,Topic,count,Top Words
0,1,"3,710","loc, gut, essen, werden, restaurant"
1,2,103,"dessert, kaffee, frühstück, kuchen, espresso"
2,3,69,"pizza, pizze, geben, chef, raum"
3,4,56,"steak, erlebnis, authentisch, fleisch, völlig"
4,5,50,"vegetarisch, vegan, sitzen, variante, vegetarier"
